## Importação das Bibliotecas

In [1]:
# Pacotes de manipulação de dados
import numpy as np
import pandas as pd

# Pacotes gráficos
import matplotlib.pyplot as plt
import seaborn as sns
from composicao_estatisticas import grafico_residuos

# Pacotes de modelagem
import statsmodels.api as sm
from scipy.stats import pearsonr

ModuleNotFoundError: No module named 'composicao_estatisticas'

## Importação e tratamento da base de dados

In [ ]:
# Carregar os dados do arquivo Excel
df = pd.read_excel('../data/base_salarios.xlsx')

In [ ]:
# Exibir o número de linhas e colunas
print('Número de linhas e colunas:')
print(df.shape)

In [ ]:
# Exibir as primeiras linhas dos dados
df.head(8)

In [ ]:
# Remover as colunas desnecessárias
columns_to_drop = ['Núm. Funcionário']
columns_to_drop = [col for col in columns_to_drop if col in df.columns]
df.drop(columns=columns_to_drop, inplace=True)


# Renomear as colunas conforme especificado
df.rename(columns={
    'Salario': 'Salario',
    'Anos de Educação Superior': 'Anos_Educ_Superior',
    'Tempo na Empresa': 'Tempo_Empresa',
    'Tempo de Experiencia em outras empresas': 'Tempo_Outras_Empresas',
    'Inglês': 'Ingles'
}, inplace=True)

## Preparação dos Dados

### Criando variáveis Dummy

In [ ]:
# Amostra de dados da base
df.sample(5, random_state=42)

In [ ]:
# Criação da variável dummy usando o get_dummies()
df2 = pd.get_dummies(df, drop_first=True)

# Converter True/False para 1/0
df2 = df2.astype(int)

df2.sample(5, random_state=42)
# df.head()

## Ajuste do Modelo de Regressão 
Usando o Statsmodels (Pacote de Modelos Estatísticos)

In [ ]:
# Visualização das 5 primeiras linhas do df2
df2.head()

In [ ]:
# Variável resposta
y = df2['Salario']

# Variáveis explicativas
df2['intercepto'] = 1 

x = df2[['intercepto',
         'Anos_Educ_Superior',
         'Tempo_Empresa',
         'Tempo_Outras_Empresas',
         'Ingles_Sim']]

In [ ]:
# Ajusta o modelo e retorna os resultados
modelo = sm.OLS(y , x)
resultado = modelo.fit()
print(resultado.summary())

**O que é o p-valor?**

O p-valor é uma medida estatística usada para ajudar a decidir se os resultados observados (as variáveis analisadas) em um experimento são significativos ou não. Para isso, comparamos os dados coletados com uma **hipótese nula**.

**O que é a hipótese nula?** 

A hipótese nula é a suposição de que não há efeito ou não há diferença entre as variáveis analisadas. Exemplo: "O tempo trabalhado em outras empresas não afeta o salário".

**Como interpretar o p-valor?**
- p < 0.05 (5%): Rejeitamos a hipótese nula. A variável tem efeito significativo.

- p >= 0.05 (5%): Não rejeitamos a hipótese nula. A variável não tem efeito significativo.

In [ ]:
from IPython.display import display, HTML

# Exibir os p-valores com cores e significância
for var, p in resultado.pvalues.items():
    p_percentual = p * 100  # Convertendo para percentual
    if p < 0.05:
        cor = '#1d8a2a'  # Verde para significante
        mensagem = "Passou no teste"
    else:
        cor = '#e76f6f'  # Vermelho para não significante
        mensagem = "Não passou no teste"
    
    resultado_texto = f"<b>{var}:</b> p-valor = {p_percentual:.2f}% <span style='color:{cor}; font-style:italic;'>({mensagem})</span>"

    # Exibindo com a cor correspondente
    display(HTML(f'<div style="font-size:14px; color:#333; padding-bottom:5px;">{resultado_texto}</div>'))

Considerando que uma variável que tem um p-valor superior a 5 não é estatisticamente significativa, **Tempo_Outras_Empresas** apresentou um p-valor acima desse limite, indicando que não há evidências suficientes para afirmar que ela afeta o salário de forma consistente. Por isso, foi excluída do modelo para focarmos nas variáveis que realmente influenciam o salário.

## Novo Ajuste do Modelo de Regressão
Sem a variável Tempo_Outras_Empresas

In [ ]:
# Variável resposta
y = df2['Salario']

# Variáveis explicativas
df2['intercepto'] = 1 

x = df2[['intercepto',
         'Anos_Educ_Superior',
         'Tempo_Empresa',
         'Ingles_Sim']]

In [ ]:
# Ajusta o modelo e retorna os resultados
modelo = sm.OLS(y , x)
resultado = modelo.fit()
print(resultado.summary())

## Equação do Modelo
Salário = $\beta_0$ + $\beta_1$ x Anos de Educação Superior + $\beta_2$ x Tempo de Empresa + $\beta_3$ x Sabe Inglês

## Interpretação do Modelo

**Intercepto ou $\beta_0$**: Salário médio de um colaborador sem educação superior, recém chegado na empresa e sem saber falar inglês é de `R$4.456,28`.<br>
**$\beta_1$** x Anos de Educação Superior: Cada ano adicional de educação superior, mantendo tempo de empresa e sabendo falar  inglês, gera um aumento médio de `R$253,59`.<br>
**$\beta_2$** x Tempo de Empresa: Cada ano adicional de tempo de empresa, mantendo os anos de educação superior e sabendo falar  inglês, gera um aumento médio de `R$61,02`.<br>
**$\beta_3$** x Sabe Inglês: Saber falar inglês, mantendo os anos de educação superior e tempo de empresa constantes, gera um aumento médio de `R$1.966,49`.

## Diagnóstico do Modelo - Análise dos Resíduos

### Teste de Homocedasticidade dos Resíduos

O que é o Teste de Homocedasticidade?

- Homocedasticidade: significa que os resíduos do modelo têm uma variância constante. Isso é bom porque indica que o modelo está funcionando bem para todos os níveis de previsão
- Heterocedasticidade: significa que a variância dos resíduos muda para diferentes níveis de previsão. Isso pode ser um problema porque sugere que o modelo pode não estar capturando toda a variabilidade dos dados de forma adequada

### Teste de White

O Teste de White é um dos testes estatísticos mais usados para verificar heterocedasticidade (ou seja, variação não constante dos resíduos). Ele complementa muito bem a análise visual do gráfico.

**Interpretação do Teste**
- Hipótese nula (H₀): Os resíduos têm variância constante (homocedasticidade).
- Hipótese alternativa (H₁): Os resíduos têm variância variável (heterocedasticidade).
- Se o p-valor < 0.05, rejeitamos H₀ → há evidência de heterocedasticidade.

In [ ]:
# Homocedasticidade (boa!): os pontos estão espalhados aleatoriamente ao redor da linha zero, sem formar padrões
# Heterocedasticidade (problema!): os resíduos formam um “funil” (abrindo ou fechando) ou algum padrão específico

grafico_residuos(resultado)

In [ ]:
from statsmodels.stats.diagnostic import het_white
labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']

white_test = het_white(resultado.resid_pearson, resultado.model.exog)
print(dict(zip(labels, white_test)))

In [ ]:
from statsmodels.stats.diagnostic import het_white
from IPython.display import display, HTML

# Rodar o teste de White
labels = ['LM Statistic', 'LM-Test p-value', 'F-Statistic', 'F-Test p-value']
white_test = het_white(resultado.resid_pearson, resultado.model.exog)
white_dict = dict(zip(labels, white_test))

# Exibir resultados formatados
html_resultado = "<h4>📊 Teste de White para Homocedasticidade</h4><ul style='font-size:14px;'>"

for key in ['LM-Test p-value', 'F-Test p-value']:
    pval = white_dict[key]
    passou = pval >= 0.05
    cor = 'green' if passou else 'red'
    status = "✅ Homocedasticidade (variância constante)" if passou else "❗ Heterocedasticidade detectada"
    html_resultado += f"<li><strong>{key}</strong>: <span style='color:{cor}'>{pval:.4f}</span> → {status}</li>"

html_resultado += "</ul><p style='font-size:12px;'>* Considerando nível de significância de 5%.</p>"

display(HTML(html_resultado))

### Teste de Normalidade dos Resíduos

In [ ]:
# Construção de histograma para avaliar a distribuição dos resíduos
sns.histplot(resultado.resid_pearson, kde=True)